In [1]:
def main():
    # 호스풀 알고리즘을 이용한 문자열 매치.
    print('# 호스풀 알고리즘을 이용한 문자열 매치.')

    print('# 연습문제 14번.')
    string = 'I_LOVE_BANANA_YOU_LIKE_APPLE_AND_MANGO'
    contained_target = 'BANANA'
    not_contained_target = 'BAMAMA'

    contained_result = horspool(string, contained_target)
    not_contained_result = horspool(string, not_contained_target)

    print(f'Word \'{contained_target}\' starts at {contained_result} in {string}, where the char at is {string[contained_result]}.')
    print(f'Word \'{not_contained_target}\' is not included. (result: {not_contained_result})\n')

    print('# 연습문제 15번.')
    nucleic_sequence = 'TTATAGATCTCGTATTCTTTTATAGATCTCCTATTCTT'
    target_nucleic_sequence = 'TCCTATTCTT'

    nucleic_result = horspool(nucleic_sequence, target_nucleic_sequence)

    print(f'Nucleic Sequence \'{target_nucleic_sequence}\' starts at {nucleic_result} in {nucleic_sequence}, where the nucleobase at is {nucleic_sequence[nucleic_result]}.\n')

    # 배낭 채우기 문제.
    print('# 배낭 채우기 문제.')

    items = [
        {'weight': 2, 'value':  60},
        {'weight': 5, 'value': 100},
        {'weight': 8, 'value': 190},
        {'weight': 4, 'value': 120},
        {'weight': 7, 'value': 200},
        {'weight': 6, 'value': 150}
    ]

    num_items = len(items)
    weight_max = 18

    result = knapsack_with_dp(items, num_items, weight_max)

    print(f'Number of items: {num_items}, Maximum capacity of knapsack: {weight_max}')
    print(f'Maximum value can be carried: {result}')

In [2]:
# 호스풀 알고리즘 시프트 테이블 생성.
def generate_shift_table(target):
    shift_table = [0 for _ in range(128)]       # 아스키 코드 문자를 기준으로 하므로 128개 공간 할당.

    # 타킷 문자열에 대해 반복.
    for index in range(len(target)):
        target_char = target[index]
        shift_table[ord(target_char)] = len(target) - 1 - index     # 타깃 문자열을 이동해야 하는 길이 계산.

    return shift_table

# 호스풀 알고리즘 주요 로직.
def horspool(string, target):
    shift_table = generate_shift_table(target)      # 시프트 테이블 생성.
    
    # 문자열 매칭을 수행할 문자열에 대해 반복.
    index = 0
    while index < len(string) - len(target) + 1:
        # 원본 문자열의 인덱스와 타깃 문자열의 인덱스를 각각 추적.
        string_index = index + len(target) - 1
        target_index = len(target) - 1

        # 각 추적에서 문자열의 뒤에서부터 탐색.
        # 문자열이 매치되지 않을 때가지 반복.
        while string[string_index] == target[target_index]:
            if target_index == 0:
                return index

            string_index -= 1
            target_index -= 1

        # 매치되지 않는 원본 문자열의 문자에 대해 시프트 테이블을 참조하여 인덱스 증가분 계산.
        index_incremental = shift_table[ord(string[string_index])] - (len(target) - target_index)
        index += max(index_incremental, 1)

    return -1

In [3]:
# 동적 계획법을 활용한 배낭 채우기 문제.
def knapsack_with_dp(items, N, K):
    items.insert(0, {'weight': 0, 'value': 0})      # 리스트 연산의 편의를 위해 인덱스를 1부터 시작하도록 0번째 인덱스에 패딩 삽입.
    table = [[0 for _ in range(K + 1)] for _ in range(N + 1)]
    return __knapsack_with_dp(items, N, K, table)

# 동적 계획법을 활용한 배낭 채우기 문제, 순환 부분.
def __knapsack_with_dp(items, item_index, current_weight, dp):
    # 베이스 케이스.
    # 물건을 선정하지 않았거나 무게가 0인 경우. (아직 물건을 담지 않은 경우)
    if item_index == 0 or current_weight == 0:
        return 0

    # dp에 값 존재 시 dp값 반환.
    if dp[item_index][current_weight] != 0:
        return dp[item_index][current_weight]

    # 현재 물건을 담을 수 없는 상황이면 해당 물건을 넣지 않은 경우 반환.
    if items[item_index].get('weight') > current_weight:
        return __knapsack_with_dp(items, item_index - 1, current_weight, dp)

    # 현재 물건을 담을 수 있는 경우에 두 가지 경우를 고려한다.

    # 현재 물건을 담지 않는 경우.
    val_without = __knapsack_with_dp(items, item_index - 1, current_weight, dp)
    # 현재 물건을 담는 경우.
    val_with = items[item_index].get('value') + __knapsack_with_dp(items, item_index - 1, current_weight - items[item_index].get('weight'), dp)

    # 동적 계획법을 적용하기 위해 dp 테이블에 값 저장.
    dp[item_index][current_weight] = max(val_without, val_with)

    return dp[item_index][current_weight]

#### **# 배낭 채우기 문제의 시간 복잡도 분석**  

##### # 분할정복 알고리즘의 시간 복잡도  
분할정복 알고리즘의 경우 현재 아이템의 인덱스 직전의 아이템에 대해 재귀 호출을 수행한다.  
즉, 이를 순환 관계식으로 나타낼 수 있다. 그런데 이때 변화하는 변수가 아이템 인덱스와 무게가 있다.  
이 둘을 모두 고려하면 관계식이 복잡해지므로 아이템 인덱스에 대해서만 고려해보자.  
무게 같은 경우 현재 무게와 현재 무게보다 1 작은 무게만 고려하므로 순환 호출의 개수가 2로 한정되기 때문이다.  

이를 생각해보면 순환 관계식은 다음과 같다. (단, n은 아이템의 개수)  
$$T(n) = 2\ T(n - 1)$$  

위의 식을 정리하면 아래와 같다.  
$$T(n) = 2\ T(n - 1) = 2^m\ T(n - m)$$  
$$→2^n\ T(0)$$  

이때 n이 0인 베이스 케이스에서의 연산은 0을 반환하므로 이때의 시간 복잡도는 상수이다.  
따라서 구하고자 하는 시간 복잡도는 다음과 같다.  
$$T(n)→O(2^n)$$  

한편, 이때의 공간 복잡도는 순환 호출의 스택을 고려하면 다음과 같다. (단, S(n)은 아이템 n개에 대한 공간 복잡도)  
$$S(n)→O(2^n)$$  

##### # 타뷸레이션을 활용한 분할정복 알고리즘의 시간 복잡도  
타뷸레이션을 활용할 때는 nw 크기의 이차원 배열을 활용해 연산을 수행하게 된다. (단, w는 배낭의 최대 수용 무게)  

(0, 0)인덱스부터 (n, w)인덱스까지 순차적으로 연산을 수행하므로 구하고자 하는 시간 복잡도는 다음과 같다.  
$$T(n)→O(nw)$$  

한편, 이때의 공간 복잡도는 nw크기의 배열을 사용하므로 다음과 같다.  
$$S(n)→O(nw)$$  

##### # 메모이제이션을 활용한 분할정복 알고리즘의 시간 복잡도  
분할정복 알고리즘에서의 시간 복잡도가 밑이 2인 지수 함수의 형태로 나온 이유는 순환 호출을 두 번씩 수행하기 때문이다.  
순환 호출을 두 번씩 수행하면 이진 트리의 형태로 순환이 진행되며 이때 트리의 개수는 두 배씩 증가하므로 밑이 2인 지수 형태의 시간 복잡도가 나타난다.  

그런데 메모이제이션을 활용하면 최초에 수행되는 연산 외에는 순환 호출을 베이스 케이스까지 갈 필요가 없으므로 최초 n번의 순환 이후에는 다시 순환할 이유가 없다.  
따라서 구하고자 하는 시간 복잡도는 다음과 같다.  
$$T(n)→O(n)$$  

그런데 이때 각 아이템 인덱스에 대한 무게 요소를 고려해야 한다.  
각 아이템 인덱스에 대해 무게 요소를 고려하므로 둘을 곱해주면 된다.  
따라서 구하고자 하는 시간 복잡도는 다음과 같다. (단, w는 배낭의 최대 수용 무게)  
$$T(n)→O(nw)$$  

한편, 메모이제이션을 구현하기 위해 타뷸레이션과 비슷하게 nw 크기의 배열이 필요하다.  
따라서 알고리즘의 공간 복잡도는 다음과 같다.  
$$S(n)→O(nw)$$  

그런데 메모이제이션의 경우 타뷸레이션과 달리 순환 호출을 제거한 것이 아니므로 실제 사용하는 메모리는 타뷸레이션보다 크다.  

##### # 결론
타뷸레이션과 메모이제이션 모두 시간 복잡도는 동일한 수준으로 감소된다.  
그러나 타뷸레이션은 순환 구조를 제거함으로 공간 복잡도에서 타뷸레이션이 유리하다.  

In [4]:
if __name__ == '__main__':
    main()

# 호스풀 알고리즘을 이용한 문자열 매치.
# 연습문제 14번.
Word 'BANANA' starts at 7 in I_LOVE_BANANA_YOU_LIKE_APPLE_AND_MANGO, where the char at is B.
Word 'BAMAMA' is not included. (result: -1)

# 연습문제 15번.
Nucleic Sequence 'TCCTATTCTT' starts at 28 in TTATAGATCTCGTATTCTTTTATAGATCTCCTATTCTT, where the nucleobase at is T.

# 배낭 채우기 문제.
Number of items: 6, Maximum capacity of knapsack: 18
Maximum value can be carried: 480
